# Neural Collaborative Filtering

In [185]:
import torch
import torch.nn as nn
import numpy as np

# ユーザー数、アイテム数、評価値の範囲
num_users = 50
num_items = 50
max_rating = 5
min_rating = 1

# データセットの生成
ratings = np.random.randint(min_rating, max_rating+1, size=(num_users, num_items))
ratings = (ratings-1)/4

# ネットワークの定義
class NCF(nn.Module):
    def __init__(self, num_users, num_items, factor_num=5, layers=[10,5]):
        super(NCF, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.factor_num = factor_num
        self.layers = layers

        self.embedding_user = nn.Embedding(num_embeddings=self.num_users, embedding_dim=5)
        self.embedding_item = nn.Embedding(num_embeddings=self.num_items, embedding_dim=5)

        self.fc_layers = nn.ModuleList()
        for idx, (in_size, out_size) in enumerate(zip(self.layers[:-1], self.layers[1:])):
            self.fc_layers.append(nn.Linear(in_size, out_size))

        self.affine_output = nn.Linear(in_features=self.layers[-1], out_features=1)
        self.logistic = nn.Sigmoid()

    def forward(self, user_indices, item_indices):
        user_embedding = self.embedding_user(user_indices)
        item_embedding = self.embedding_item(item_indices)
        vector = torch.cat([user_embedding, item_embedding], dim=-1)  # the concat latent vector
        for idx, _ in enumerate(range(len(self.fc_layers))):
            vector = self.fc_layers[idx](vector)
            vector = nn.ReLU()(vector)
        logits = self.affine_output(vector)
        rating = self.logistic(logits)
        return rating


In [199]:
model = NCF(num_users, num_items)
loss_function = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [202]:
losses = []
for epoch in range(30):
    model.train() # Enable dropout (if have).
    
    epoch_loss = 0.0
    for user_index in range(num_users):
        for item_index in range(num_items):
            rating = float(ratings[user_index, item_index])
            label = torch.FloatTensor([[rating]])
        
            optimizer.zero_grad()
            user = torch.LongTensor([user_index])
            item = torch.LongTensor([item_index])
            prediction = model(user, item)
            
            loss = loss_function(prediction, label)
            loss.backward()
            optimizer.step()
            epoch_loss+=loss.item()
    
    losses.append(epoch_loss)
    model.eval()

In [203]:
losses

[1729.9323657751083,
 1729.5330923199654,
 1729.1415237784386,
 1728.7692486643791,
 1728.3915739655495,
 1727.9939889907837,
 1727.603432893753,
 1727.2323777079582,
 1726.850794017315,
 1726.4513570070267,
 1726.0422974526882,
 1725.6258172988892,
 1725.1787912845612,
 1724.7289662361145,
 1724.2784088551998,
 1723.8056592345238,
 1723.3448340594769,
 1722.8975209593773,
 1722.478587925434,
 1722.0650717020035,
 1721.6422416567802,
 1721.2343438267708,
 1720.799686074257,
 1720.3994975090027,
 1719.9930233955383,
 1719.5862289071083,
 1719.1797151565552,
 1718.7649176716805,
 1718.3551274240017,
 1717.9440604746342]